# This is the AWS EC2 Container Service Demo
This notebook assumes you have created a container (here called cloudbook) by following the instructions in the book text.   here are the important points you must do first.

## Cluster set  up.  
Next you must create ec2 instances to populate the cluster.   
Follow the instructions here 

If you need the instances to be visible to the world at certain ports, you will need a new security group.   Go to your resources in the ec2 management council.  Click on security group.  Look at your default security group.  Add inbound NFS port  2049 by selecting Edit inbound rules from the "actions" tab.   If you need extra inbound ports for your services you may then want to copy to new and add them there and give the new security group a name and description.  I created one with ports 8000-9000 open and give it the name ecs-security-group.


http://docs.aws.amazon.com/AmazonECS/latest/developerguide/launch_container_instance.html

Go to the console for launching instances and hit "launch instance" go to community aims and search for "amazon-ecs-optimized" and you should find  ami-1ccd1f7c if you are on us-west-2.    Select that one. Then the size you want.   T2.medium is fine.   Then go to configure launch instance details.  
Set the number of instances (I choose 4), enable auto-assign public IP.   You next need an IAM role.  Set it.   

Next go to Advanced Details at the bottom of the page.   Paste the following in the text box

------------------
#!/bin/bash

echo ECS_CLUSTER=cloudbook >> /etc/ecs/ecs.config

------------------
Next go to "add storage".   The default is fine for now.  Tag it with name like 'cbook'.   Finally configure security group.   Choose select existing security group.  Your new group should be there.  If not, then look for default and copy-to-new and add the ports you need.

Launch.   When you launch you will need to select your key pair in case you need to ssh into these new instances.

## this notebook
This notebook shows you how to interact with the container service once you have your clust.   

In [1]:
import boto3

In [2]:
client = boto3.client('ecs')

the list operators return dictionaries and we usually want to see the Arns.

In [3]:
client.list_clusters()['clusterArns']

[u'arn:aws:ecs:us-west-2:066301190734:cluster/default',
 u'arn:aws:ecs:us-west-2:066301190734:cluster/cloudbook']

In [4]:
instance_list = client.list_container_instances(cluster='cloudbook')['containerInstanceArns']

In [5]:
instance_list

[u'arn:aws:ecs:us-west-2:066301190734:container-instance/12607795-a317-4732-8e6c-5e86f67b0630',
 u'arn:aws:ecs:us-west-2:066301190734:container-instance/39158178-408e-4ff3-abc4-ed4734d70ced',
 u'arn:aws:ecs:us-west-2:066301190734:container-instance/7c2516f0-8131-4f10-b3a5-1f8ca3a1e566',
 u'arn:aws:ecs:us-west-2:066301190734:container-instance/e578f40c-0d4f-46f8-937d-c36e77ba6aa6']

In [6]:
ec2instances = [ client.describe_container_instances(
        cluster='cloudbook',
        containerInstances=[instance]
    )[u'containerInstances'][0][u'ec2InstanceId'] for instance in instance_list]

In [7]:
ec2 = boto3.resource('ec2')

In [8]:
instances = ec2.instances.filter( Filters=[{'Name': 'instance-id', 'Values': ec2instances}])
for instance in instances:
   print(instance.id, instance.instance_type, instance.image_id, instance.public_ip_address)

('i-00dd7298fca2948df', 't2.medium', 'ami-1ccd1f7c', '54.201.144.148')
('i-057e2913cc97af6d0', 't2.medium', 'ami-1ccd1f7c', '54.186.99.177')
('i-0f46d62b305a8487c', 't2.medium', 'ami-1ccd1f7c', '54.214.228.63')
('i-089a5074ce074fc49', 't2.medium', 'ami-1ccd1f7c', '54.218.107.231')


In [9]:
client.list_task_definition_families()['families']

[u'awsque',
 u'container-try-2',
 u'defx',
 u'esciencebook-task-def',
 u'escinewer',
 u'finaltry',
 u'finaltryagain',
 u'jupyter',
 u'newsynth',
 u'predict',
 u'predictor-new',
 u'table-manager']

In [41]:
client.list_task_definitions(familyPrefix='predict')['taskDefinitionArns']

[u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:1',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:2',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:3',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:4',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:5',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:6']

In [42]:
client.describe_task_definition(taskDefinition='predict:6')['taskDefinition']

{u'containerDefinitions': [{u'cpu': 20,
   u'environment': [],
   u'essential': True,
   u'image': u'dbgannon/predict2',
   u'memoryReservation': 400,
   u'mountPoints': [],
   u'name': u'predict',
   u'portMappings': [],
   u'volumesFrom': []}],
 u'family': u'predict',
 u'networkMode': u'bridge',
 u'requiresAttributes': [{u'name': u'com.amazonaws.ecs.capability.docker-remote-api.1.21'}],
 u'revision': 6,
 u'status': u'ACTIVE',
 u'taskDefinitionArn': u'arn:aws:ecs:us-west-2:066301190734:task-definition/predict:6',
 u'volumes': []}

 The following will crecate a task revision of task definition predictor-new. that should be version 6, because there were 5 previous versions (not shown here). 

In [33]:
response = client.register_task_definition(
    family='predictor-new',
    networkMode='bridge',
    containerDefinitions=[
        {
            'name': 'predictor-new',
            'image': 'dbgannon/predictor-new',
            'cpu': 20,
            'memoryReservation': 400,
            'essential': True,
        },
    ],
 )


In [34]:
response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '489',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 11 Oct 2016 18:29:42 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'ad977843-8fe0-11e6-b135-efdf65d29c30'},
  'HTTPStatusCode': 200,
  'RequestId': 'ad977843-8fe0-11e6-b135-efdf65d29c30',
  'RetryAttempts': 0},
 u'taskDefinition': {u'containerDefinitions': [{u'cpu': 20,
    u'environment': [],
    u'essential': True,
    u'image': u'dbgannon/predictor-new',
    u'memoryReservation': 400,
    u'mountPoints': [],
    u'name': u'predictor-new',
    u'portMappings': [],
    u'volumesFrom': []}],
  u'family': u'predictor-new',
  u'networkMode': u'bridge',
  u'requiresAttributes': [{u'name': u'com.amazonaws.ecs.capability.docker-remote-api.1.21'}],
  u'revision': 6,
  u'status': u'ACTIVE',
  u'taskDefinitionArn': u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor-new:6',
  u'volumes': []}}

In [35]:
response = client.register_task_definition(
    family='table-manager',
    networkMode='bridge',
    containerDefinitions=[
        {
            'name': 'table-manager',
            'image': 'dbgannon/table-service-bottle',
            'cpu': 400,
            'memoryReservation': 200,
            'portMappings': [
                {
                    'containerPort': 8050,
                    'hostPort': 8050,
                    'protocol': 'tcp'
                },
            ],

            'essential': True,
        },
    ],
 )


In [36]:
response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '554',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 11 Oct 2016 18:29:57 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'b65fb353-8fe0-11e6-b135-efdf65d29c30'},
  'HTTPStatusCode': 200,
  'RequestId': 'b65fb353-8fe0-11e6-b135-efdf65d29c30',
  'RetryAttempts': 0},
 u'taskDefinition': {u'containerDefinitions': [{u'cpu': 400,
    u'environment': [],
    u'essential': True,
    u'image': u'dbgannon/table-service-bottle',
    u'memoryReservation': 200,
    u'mountPoints': [],
    u'name': u'table-manager',
    u'portMappings': [{u'containerPort': 8050,
      u'hostPort': 8050,
      u'protocol': u'tcp'}],
    u'volumesFrom': []}],
  u'family': u'table-manager',
  u'networkMode': u'bridge',
  u'requiresAttributes': [{u'name': u'com.amazonaws.ecs.capability.docker-remote-api.1.21'}],
  u'revision': 12,
  u'status': u'ACTIVE',
  u'taskDefinitionArn': u'arn:aws:ecs:us-west-2:06630119

now we are going to create 8 instances of this new service based on this task definition

In [37]:
response = client.create_service(
    cluster='cloudbook',
    serviceName='table-manager',
    taskDefinition='table-manager:12',
    desiredCount=4,   
    deploymentConfiguration={
        'maximumPercent': 100,
        'minimumHealthyPercent': 50
    }
)

In [38]:
response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '993',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 11 Oct 2016 18:30:12 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'bf6d34a7-8fe0-11e6-b135-efdf65d29c30'},
  'HTTPStatusCode': 200,
  'RequestId': 'bf6d34a7-8fe0-11e6-b135-efdf65d29c30',
  'RetryAttempts': 0},
 u'service': {u'clusterArn': u'arn:aws:ecs:us-west-2:066301190734:cluster/cloudbook',
  u'createdAt': datetime.datetime(2016, 10, 11, 11, 30, 12, 868000, tzinfo=tzlocal()),
  u'deploymentConfiguration': {u'maximumPercent': 100,
   u'minimumHealthyPercent': 50},
  u'deployments': [{u'createdAt': datetime.datetime(2016, 10, 11, 11, 30, 12, 868000, tzinfo=tzlocal()),
    u'desiredCount': 4,
    u'id': u'ecs-svc/9223370560644162930',
    u'pendingCount': 0,
    u'runningCount': 0,
    u'status': u'PRIMARY',
    u'taskDefinition': u'arn:aws:ecs:us-west-2:066301190734:task-definition/table-manager:12',
    u'updatedAt': d

In [39]:
response = client.create_service(
    cluster='cloudbook',
    serviceName='predictor-new',
    taskDefinition='predictor-new:6',
    desiredCount=1,   
    deploymentConfiguration={
        'maximumPercent': 100,
        'minimumHealthyPercent': 50
    }
)

In [20]:
response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '993',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 11 Oct 2016 17:28:54 GMT',
   'server': 'Server',
   'x-amzn-requestid': '2ed68fbf-8fd8-11e6-b135-efdf65d29c30'},
  'HTTPStatusCode': 200,
  'RequestId': '2ed68fbf-8fd8-11e6-b135-efdf65d29c30',
  'RetryAttempts': 0},
 u'service': {u'clusterArn': u'arn:aws:ecs:us-west-2:066301190734:cluster/cloudbook',
  u'createdAt': datetime.datetime(2016, 10, 11, 10, 28, 54, 339000, tzinfo=tzlocal()),
  u'deploymentConfiguration': {u'maximumPercent': 100,
   u'minimumHealthyPercent': 50},
  u'deployments': [{u'createdAt': datetime.datetime(2016, 10, 11, 10, 28, 54, 339000, tzinfo=tzlocal()),
    u'desiredCount': 8,
    u'id': u'ecs-svc/9223370560647841459',
    u'pendingCount': 0,
    u'runningCount': 0,
    u'status': u'PRIMARY',
    u'taskDefinition': u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor-new:3',
    u'updatedAt': da

In [14]:
client.list_services( cluster='cloudbook')['serviceArns']

[u'arn:aws:ecs:us-west-2:066301190734:service/predictor5',
 u'arn:aws:ecs:us-west-2:066301190734:service/awsque1',
 u'arn:aws:ecs:us-west-2:066301190734:service/table-manager',
 u'arn:aws:ecs:us-west-2:066301190734:service/predictor6',
 u'arn:aws:ecs:us-west-2:066301190734:service/synth9']

In [79]:
response = client.register_task_definition(
    family='newsynth',
    networkMode='bridge',
    containerDefinitions=[
        {
            'name': 'synthcontainer3',
            'image': '066301190734.dkr.ecr.us-west-2.amazonaws.com/escibook',
            'cpu': 20,
            'memory': 400,
            'memoryReservation': 123,
            'portMappings': [
                {
                    'containerPort': 8000,
                    'hostPort': 8010,
                    'protocol': 'tcp'
                },
            ],
            'essential': True,
        },
    ],
 )
